In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
from sklearn.model_selection import GridSearchCV
from  sklearn.tree  import  DecisionTreeClassifier
from  sklearn.ensemble  import  RandomForestClassifier , VotingClassifier
from  sklearn.linear_model  import  LogisticRegression
from  sklearn.metrics  import  accuracy_score , roc_curve , auc , f1_score, confusion_matrix, classification_report
from  sklearn.preprocessing  import  LabelEncoder , MinMaxScaler
from  sklearn  import svm #SVC , LinearSVC
from sklearn.neighbors import NearestNeighbors
#draw roc cuver
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from imblearn.metrics import (geometric_mean_score, make_index_balanced_accuracy)
from imblearn.metrics import classification_report_imbalanced
import classifier
import time

dt = unsw.read_data_sets()

In [ ]:
start = time.time()
mb_size = 100
X_dim = dt.train.features.shape[1]
y_dim = dt.train.labels.shape[1]
z_dim = y_dim
h_dim = 100
eps = 1e-8
lr = 1e-3
d_steps = 3
X_train = dt.train.features
Y_train = dt.train.labels

X_test = dt.test.features
Y_test = dt.test.labels

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)


X = tf.placeholder(tf.float32, shape=[None, X_dim])
y = tf.placeholder(tf.float32, shape=[None, y_dim])
z = tf.placeholder(tf.float32, shape=[None, z_dim])

G_W1 = tf.Variable(xavier_init([z_dim + y_dim, h_dim]))
G_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

G_W2 = tf.Variable(xavier_init([h_dim, h_dim]))
G_b2 = tf.Variable(tf.zeros(shape=[h_dim]))


G_W5 = tf.Variable(xavier_init([h_dim, X_dim]))
G_b5 = tf.Variable(tf.zeros(shape=[X_dim]))


def generator(z, c):
    inputs = tf.concat(axis=1, values=[z, c])
    G_h1 = tf.nn.relu(tf.matmul(inputs, G_W1) + G_b1)
    G_h2 = tf.nn.relu(tf.matmul(G_h1, G_W2) + G_b2)
    G_log_prob = tf.matmul(G_h2, G_W5) + G_b5
    G_prob = tf.nn.sigmoid(G_log_prob)
    return G_prob


D_W1 = tf.Variable(xavier_init([X_dim, h_dim]))
D_b1 = tf.Variable(tf.zeros(shape=[h_dim]))

D_W2 = tf.Variable(xavier_init([h_dim, h_dim]))
D_b2 = tf.Variable(tf.zeros(shape=[h_dim]))

D_W2_gan = tf.Variable(xavier_init([h_dim, 1]))
D_b2_gan = tf.Variable(tf.zeros(shape=[1]))
D_W2_aux = tf.Variable(xavier_init([h_dim, y_dim]))
D_b2_aux = tf.Variable(tf.zeros(shape=[y_dim]))


def discriminator(X):
    D_h1 = tf.nn.relu(tf.matmul(X, D_W1) + D_b1)
    D_h2 = tf.nn.relu(tf.matmul(D_h1, D_W2) + D_b2)
         
    out_gan = tf.nn.sigmoid(tf.matmul(D_h2, D_W2_gan) + D_b2_gan)
    out_aux = tf.matmul(D_h2, D_W2_aux) + D_b2_aux
    return out_gan, out_aux


theta_G = [G_W1, G_W2, G_W5, G_b1, G_b2, G_b5]
theta_D = [D_W1,D_W2, D_W2_gan, D_W2_aux, D_b1, D_b2, D_b2_gan, D_b2_aux]


def sample_z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])


def cross_entropy(logit, y):
    return -tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logit, labels=y))


G_sample = generator(z, y)

D_real, C_real = discriminator(X)
D_fake, C_fake = discriminator(G_sample)

# Cross entropy aux loss
C_loss = cross_entropy(C_real, y) + cross_entropy(C_fake, y)

# GAN D loss
D_loss = tf.reduce_mean(tf.log(D_real + eps) + tf.log(1. - D_fake + eps))
DC_loss = -(D_loss + C_loss)

# GAN's G loss
G_loss = tf.reduce_mean(tf.log(D_fake + eps))
GC_loss = -(G_loss + C_loss)

D_solver = (tf.train.AdamOptimizer(learning_rate=lr)
            .minimize(DC_loss, var_list=theta_D))
G_solver = (tf.train.AdamOptimizer(learning_rate=lr)
            .minimize(GC_loss, var_list=theta_G))


In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())


for it in range(500000):
    X_mb, y_mb = dt.train.next_batch(mb_size)
    z_mb = sample_z(mb_size, z_dim)

    _, DC_loss_curr = sess.run(
        [D_solver, DC_loss],
        feed_dict={X: X_mb, y: y_mb, z: z_mb}
    )

    _, GC_loss_curr = sess.run(
        [G_solver, GC_loss],
        feed_dict={X: X_mb, y: y_mb, z: z_mb}
    )

    if it % 1000 == 0:   
        print('Iter: {}; DC_loss: {:.4}; GC_loss: {:.4}'
              .format(it, DC_loss_curr, GC_loss_curr))
        
stop = time.time()
print ("thoi gian chay")
print (stop-start)

In [ ]:
n_samples = 10000
c = np.zeros(shape=[n_samples, y_dim])
idx = 3
c[range(n_samples), idx] = 1.
X_gen0 = sess.run(G_sample, feed_dict={z: np.random.randn(n_samples, z_dim), y: c})
Y_gen0 = c

n_samples = 10000
y = np.zeros(shape=[n_samples, y_dim])
idx = 4
y[:, idx] = 1.
X_gen1 = sess.run(G_sample, feed_dict={z: np.random.randn(n_samples, z_dim), y: y})
Y_gen1 = y


In [ ]:
#combine Generated samples and Original Samples
#nslkdd
#X = np.concatenate ((X_train, X_gen0, X_gen1), axis = 0)
#Y = np.concatenate ((Y_train, Y_gen0, Y_gen1), axis = 0)

#unsw
#X = np.concatenate ((X_train, X_gen0, X_gen1, X_gen2, X_gen3, X_gen4, X_gen5, X_gen6, X_gen7, X_gen8, X_gen9), axis = 0)
#Y = np.concatenate ((Y_train, Y_gen0, Y_gen1,Y_gen2, Y_gen3, Y_gen4, Y_gen5, Y_gen6, Y_gen7, Y_gen8, Y_gen9), axis = 0)

#ids2017
X = np.concatenate ((X_train, X_gen0, X_gen1), axis = 0)
Y = np.concatenate ((Y_train, Y_gen0, Y_gen1), axis = 0)
from collections import Counter
Y_count = np.argmax(Y, axis=1)
print('Resampled dataset shape {}'.format(Counter(Y_count)))

s = np.arange(X.shape[0])
np.random.shuffle(s)
X = X[s]
Y = Y[s]

print ("-------------------------ACGAN----------------------")


print ("SVM")
sv_gan, svm_fpr_gan, svm_tpr_gan, svm_roc_gan = classifier.svm_svc(X, Y, X_test,Y_test)
print ("Decision tree")
dt_gan, dt_fpr_gan, dt_tpr_gan, dt_roc_gan = classifier.decisiontree(X, Y, X_test,Y_test)
print ("Random forest")
rf_gan, rf_fpr_gan, rf_tpr_gan, rf_roc_gan = classifier.randomforest(X, Y, X_test,Y_test)

In [ ]:
n_samples = 35000
c = np.zeros(shape=[n_samples, y_dim])
idx = 0
c[range(n_samples), idx] = 1.
X_gen0 = sess.run(G_sample, feed_dict={z: np.random.randn(n_samples, z_dim), y: c})
Y_gen0 = c

n_samples = 35000
y = np.zeros(shape=[n_samples, y_dim])
idx = 1
y[:, idx] = 1.
X_gen1 = sess.run(G_sample, feed_dict={z: np.random.randn(n_samples, z_dim), y: y})
Y_gen1 = y

n_samples = 33000
y = np.zeros(shape=[n_samples, y_dim])
idx = 2
y[:, idx] = 1.
X_gen2 = sess.run(G_sample, feed_dict={z: np.random.randn(n_samples, z_dim), y: y})
Y_gen2 = y

n_samples = 4000
y = np.zeros(shape=[n_samples, y_dim])
idx = 3
y[:, idx] = 1.
X_gen3 = sess.run(G_sample, feed_dict={z: np.random.randn(n_samples, z_dim), y: y})
Y_gen3 = y

n_samples = 21000
y = np.zeros(shape=[n_samples, y_dim])
idx = 4
y[:, idx] = 1.
X_gen4 = sess.run(G_sample, feed_dict={z: np.random.randn(n_samples, z_dim), y: y})
Y_gen4 = y

n_samples = 27000
y = np.zeros(shape=[n_samples, y_dim])
idx = 7
y[:, idx] = 1.
X_gen5 = sess.run(G_sample, feed_dict={z: np.random.randn(n_samples, z_dim), y: y})
Y_gen5 = y

n_samples = 36000
y = np.zeros(shape=[n_samples, y_dim])
idx = 8
y[:, idx] = 1.
X_gen6 = sess.run(G_sample, feed_dict={z: np.random.randn(n_samples, z_dim), y: y})
Y_gen6 = y

n_samples = 37000
y = np.zeros(shape=[n_samples, y_dim])
idx = 9
y[:, idx] = 1.
X_gen7 = sess.run(G_sample, feed_dict={z: np.random.randn(n_samples, z_dim), y: y})
Y_gen7 = y

n_samples = 19000
y = np.zeros(shape=[n_samples, y_dim])
idx = 5
y[:, idx] = 1.
X_gen8 = sess.run(G_sample, feed_dict={z: np.random.randn(n_samples, z_dim), y: y})
Y_gen8 = y

n_samples = 20000
y = np.zeros(shape=[n_samples, y_dim])
idx = 3
y[:, idx] = 1.
X_gen9 = sess.run(G_sample, feed_dict={z: np.random.randn(n_samples, z_dim), y: y})
Y_gen9 = y



